In [1]:
from langchain.agents import initialize_agent, Tool, AgentType,tool
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import requests
import os

In [2]:
# Configure the LLM Model
apikey = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite',
                             api_key=apikey)

In [3]:
# Python Coding TOOL
@tool('Python Coading Tool')
def run_python(code: str):
    """Execute Python code and return result."""
    try:
        # Create a local dictionary for safe execution
        local_vars = {}
        exec(code,locals=local_vars)
        return str(local_vars) if local_vars else "Code executed successfully"
    except Exception as e:
        return f"Error: {e}"

In [4]:
# Google Web Search Tool

api = os.getenv('API_KEY')

@tool('Tool for web search')
def google_search(query: str):
    """Perform Google Custom Search and return top result snippet."""
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": api,
        "cx": '84da54018bf7d48ed',
        "q": query,
        "num": 1
    }
    response = requests.get(url, params=params).json()
    items = response.get("items", [])
    if items:
        return items[0].get("snippet", "No snippet available.")
    return "No results found."

In [5]:
# Creating Tools
coading_tool = Tool(name='CoadingTool',
                    func=run_python,
                    description='This runs the python code.')

web_search_tool = Tool(name='WebSearchTool',
                       func=google_search,
                       description='This too performs web search')

tool_list = [coading_tool,web_search_tool]

In [6]:
# Configure agent
agent = initialize_agent(tools=tool_list,
                         llm=llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

C:\Users\gaura\AppData\Local\Temp\ipykernel_5180\2147114073.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools=tool_list,


In [7]:
# Lets Run a response on Agent
response1 = 'use run_python to write a python code to add 2 in every element of list a=[3,8,4,6] and return the new list'
variables = agent.run(response1)

C:\Users\gaura\AppData\Local\Temp\ipykernel_5180\1584938569.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  variables = agent.run(response1)




> Entering new AgentExecutor chain...


I need to write a Python code to add 2 to each element of the given list `a = [3, 8, 4, 6]` and return the modified list. The `CoadingTool` can be used to execute Python code.

I will use a list comprehension to achieve this. The code will iterate through each element in the list `a`, add 2 to it, and create a new list with the results.
Action: CoadingTool
Action Input: print([x + 2 for x in [3, 8, 4, 6]])[5, 10, 6, 8]

Observation: Code executed successfully
Thought:The Python code has been executed and it produced the new list with 2 added to each element. I can now provide the final answer.
Final Answer: [5, 10, 6, 8]

> Finished chain.


In [8]:
variables

'[5, 10, 6, 8]'

In [9]:
response2 = 'Use Google_Search tool to get and summarize top 2 news from Delhi NCR today'
news = agent.run(response2)




> Entering new AgentExecutor chain...
I need to find the top 2 news from Delhi NCR today using the Google Search tool and then summarize them.
Action: WebSearchTool
Action Input: top 2 news Delhi NCR today
Observation: ... 2 months, faked UN posts for favours — arrested in Agra ... Delhi CM lays foundation for foot over bridge for soldiers: 'Their comfort our top priority'.
Thought:The search results provide a few news items. I need to identify the top 2 most relevant and recent ones specifically for Delhi NCR. The first result mentions an arrest in Agra which might not be directly related to Delhi NCR news unless it has a strong connection. The second result is about the Delhi CM laying the foundation for a foot overbridge, which is clearly relevant to Delhi NCR. I will look for another distinct news item that is also relevant to Delhi NCR and then summarize both.
Action: WebSearchTool
Action Input: latest news Delhi NCR today
Observation: Delhi News Today: Read Latest New Delhi New

In [10]:
response2

'Use Google_Search tool to get and summarize top 2 news from Delhi NCR today'